In [32]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [33]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=20)

In [35]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [36]:
X_test_scaled

,age,affordibility
13,0.29,0
4,0.46,1
1,0.25,0
10,0.18,1
27,0.46,1
12,0.27,0


In [37]:

model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=4000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 8ms/step - loss: 0.5312 - accuracy: 0.6364
Epoch 1502/4000
1/1 [==============================] - 0s 14ms/step - loss: 0.5311 - accuracy: 0.6364
Epoch 1503/4000
1/1 [==============================] - 0s 10ms/step - loss: 0.5311 - accuracy: 0.6364
Epoch 1504/4000
1/1 [==============================] - 0s 9ms/step - loss: 0.5310 - accuracy: 0.6364
Epoch 1505/4000
1/1 [==============================] - 0s 18ms/step - loss: 0.5310 - accuracy: 0.6364
Epoch 1506/4000
1/1 [==============================] - 0s 16ms/step - loss: 0.5309 - accuracy: 0.6364
Epoch 1507/4000
1/1 [==============================] - 0s 17ms/step - loss: 0.5309 - accuracy: 0.6364
Epoch 1508/4000
1/1 [==============================] - 0s 17ms/step - loss: 0.5308 - accuracy: 0.6364
Epoch 1509/4000
1/1 [==============================] - 0s 16ms/step - loss: 0.5308 - accuracy: 0.6364
Epoch 1510/4000
1/1 [============================

In [38]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 167ms/step


array([[0.25230592],
       [0.7361643 ],
       [0.21639265],
       [0.40682474],
       [0.7361643 ],
       [0.23387071]], dtype=float32)

In [39]:
X_test_scaled

,age,affordibility
13,0.29,0
4,0.46,1
1,0.25,0
10,0.18,1
27,0.46,1
12,0.27,0


In [40]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 252ms/step - loss: 0.4937 - accuracy: 0.8333


[0.49366819858551025, 0.8333333134651184]

In [41]:
coff, intercept = model.get_weights()
coff, intercept

(array([[5.011551 ],
        [1.2605146]], dtype=float32),
 array([-2.5397012], dtype=float32))

In [42]:
def sigmoid(x):
    import math
    return 1 / (1 + math.exp(-x))
sigmoid(40)

1.0

In [56]:
coff[0], coff[1]

(array([5.011551], dtype=float32), array([1.2605146], dtype=float32))

In [43]:
def prediction (age, affordibility):
    weighted_sum = coff[0]*age + coff[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [66]:
prediction(.46,1)

<ipython-input-42-980fcaf22305>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))


0.7361642721410068

In [67]:
def mae(y_true,y_predicted):
  total_error = 0
  for yt,yp in zip(y_true,y_predicted):
     total_error += abs(yp-yt)

  print("total error:" ,total_error)
  mae = total_error/len(y_true)
  print("mae is", mae)
  return mae






In [58]:
  mae(y_true,y_predicted)

total error: 2.5
mae is 0.5


0.5

In [59]:
np.mean(np.abs(y_predicted-y_true))

0.5

In [60]:
y_predicted

array([1, 1, 0, 0, 1])

In [61]:
log_change = 1e-15
np.log(log_change)



-34.538776394910684

In [62]:
y_predicted_new = [max(i,log_change) for i in y_predicted]
y_predicted_new

[1, 1, 1e-15, 1e-15, 1]

In [63]:
y_predicted_new = [min(i,1-log_change) for i in y_predicted_new]
y_predicted_new

[0.999999999999999, 0.999999999999999, 1e-15, 1e-15, 0.999999999999999]

In [64]:
def log_loss(y_predicted,y_true):

 y_predicted_new = [max(i,log_change) for i in y_predicted]
 y_predicted_new = [min(i,1-log_change) for i in y_predicted_new]
 y_predicted_new = np.array(y_predicted_new)
 return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [65]:
log_loss(y_predicted,y_true)

17.2696280766844

In [76]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [83]:
def gradient(age,affordibility,y_true,epochs):
  w1 = w2 = 1
  bias = 0
  lr = 0.5
  n = len(age)

  for i in range(epochs):
    weight = w1*age + w2*affordibility + bias
    y_predicted = sigmoid_numpy(weight)
    loss = log_loss(y_predicted,y_true)

    w1d = (1/n)*(np.dot((y_predicted - y_true),np.transpose(age)))
    w2d = (1/n)*(np.dot((y_predicted - y_true),np.transpose(affordibility)))
    bias_d = np.mean(y_predicted - y_true)

    w1 = w1 - lr*w1d
    w2 = w2 - lr*w2d
    bias = bias - lr*bias_d

    print(f"epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")

  return w1,w2,bias

In [84]:
gradient(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000)

epoch:0, w1:0.9899044631276738, w2:0.9569405919425348, bias:-0.07899372792977674, loss:0.645879853336323
epoch:1, w1:0.9839956437241003, w2:0.9213120476683795, bias:-0.14783282323721283, loss:0.6306399722613678
epoch:2, w1:0.9818392584739325, w2:0.8924082047006104, bias:-0.20758236411882589, loss:0.6194338742775531
epoch:3, w1:0.9829714996091098, w2:0.8694378697530099, bias:-0.25936841626475815, loss:0.6112638943448379
epoch:4, w1:0.9869327312511998, w2:0.8515941187585563, bias:-0.3042992643556669, loss:0.6053096946136317
epoch:5, w1:0.9932909005876579, w2:0.8381042430709594, bias:-0.3434115278383518, loss:0.6009323224120783
epoch:6, w1:1.0016552381106507, w2:0.828260552915365, bias:-0.3776393247774238, loss:0.5976547897595172
epoch:7, w1:1.0116822714409655, w2:0.821435592496283, bias:-0.407801431258409, loss:0.5951326106822025
epoch:8, w1:1.0230764951373805, w2:0.817086248135435, bias:-0.4346007971785253, loss:0.5931233031450338
epoch:9, w1:1.035587744835964, w2:0.81475080344, bias:-0

(9.232330168126946, 1.7296917858704566, -4.634456325370888)